
Using dbutils.fs.ls command to check the file that been uploaded

In [0]:
#Using dbutils.fs.ls command to check the file that been uploaded

dbutils.fs.ls("/FileStore/tables/")

Out[1]: [FileInfo(path='dbfs:/FileStore/tables/account-models/', name='account-models/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/accounts/', name='accounts/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/accounts.zip', name='accounts.zip', size=5297592, modificationTime=1706721501000),
 FileInfo(path='dbfs:/FileStore/tables/accounts2/', name='accounts2/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations/', name='activations/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations.zip', name='activations.zip', size=8411369, modificationTime=1706718610000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2023/', name='clinicaltrial_2023/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2023-1.zip', name='clinicaltrial_2023-1.zip', size=52785750, modificationTime=1712308832000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2023.zip', n


Defining Reusable code to unzip all files

In [0]:
#Copy unzipped file
fileroot = "pharma"

dbutils.fs.cp("FileStore/tables/" + fileroot + ".zip", "file:/tmp/")

Out[149]: True

In [0]:
#import os library

import os
os.environ['fileroot'] = fileroot


In [0]:
%sh

unzip -d /tmp /tmp/$fileroot.zip

Archive:  /tmp/pharma.zip
  inflating: /tmp/pharma.csv         


In [0]:
# List the contents of the /tmp/ directory
files_in_tmp = dbutils.fs.ls("file:/tmp/")

# Print the list of files and directories in /tmp/
for file_info in files_in_tmp:
    print(file_info.path)

file:/tmp/hsperfdata_root/
file:/tmp/clinicaltrial_2023.zip
file:/tmp/systemd-private-088e693699db4ad8957720cbacbb19fd-systemd-resolved.service-aI4sZf/
file:/tmp/systemd-private-088e693699db4ad8957720cbacbb19fd-ntp.service-UFh8Hh/
file:/tmp/driver-env.sh
file:/tmp/chauffeur-env.sh
file:/tmp/.java_pid542
file:/tmp/.font-unix/
file:/tmp/pharma.csv
file:/tmp/.ICE-unix/
file:/tmp/chauffeur-daemon-params
file:/tmp/pharma.zip
file:/tmp/driver-daemon-params
file:/tmp/chauffeur-daemon.pid
file:/tmp/driver-daemon.pid
file:/tmp/.XIM-unix/
file:/tmp/.Test-unix/
file:/tmp/systemd-private-088e693699db4ad8957720cbacbb19fd-apache2.service-V9cZQe/
file:/tmp/Rserv/
file:/tmp/RtmprdSLZ1/
file:/tmp/tmp.msYPOARKSF
file:/tmp/custom-spark.conf
file:/tmp/systemd-private-088e693699db4ad8957720cbacbb19fd-systemd-logind.service-jAUPci/
file:/tmp/.X11-unix/
file:/tmp/python_lsp_logs/


In [0]:
#Make directory

dbutils.fs.mkdirs("FileStore/tables/" + fileroot)

Out[153]: True

In [0]:
#Move files

dbutils.fs.mv("file:/tmp/" + fileroot + ".csv" , "/FileStore/tables/" + fileroot , True)

Out[154]: True

In [0]:
# remove the .zip file

dbutils.fs.rm("/FileStore/tables/" + fileroot + ".zip")

dbutils.fs.ls("/FileStore/tables/" + fileroot)

Out[155]: [FileInfo(path='dbfs:/FileStore/tables/pharma/pharma.csv', name='pharma.csv', size=678999, modificationTime=1713053871000)]

In [0]:
file = "/FileStore/tables/clinicaltrial_2023/clinicaltrial_2023.csv"

dbutils.fs.head(file)

[Truncated to first 65536 bytes]
Out[9]: 'NCT Number \\t Study Title \\t Acronym \\t Study Status \\t Conditions \\t Interventions \\t Sponsor \\t Collaborators \\t Enrollment \\t Funder Type \\t Study Type \\t Study Design,Start Date,Completion Date\r\n"NCT03630471 \\t Effectiveness of a Problem-solving Intervention for Common Adolescent Mental Health Problems in India \\t PRIDE \\t COMPLETED \\t Mental Health Issue (E.G., Depression, Psychosis, Personality Disorder, Substance Abuse) \\t BEHAVIORAL: PRIDE \'Step 1\' problem-solving intervention|BEHAVIORAL: Enhanced usual care \\t Sangath \\t Harvard Medical School (HMS and HSDM)|London School of Hygiene and Tropical Medicine \\t 250 \\t OTHER \\t INTERVENTIONAL \\t Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (INVESTIGATOR, OUTCOMES_ASSESSOR)|Primary Purpose: TREATMENT",20 August 2018,28 February 2019\r\n"NCT05992571 \\t Oral Ketone Monoester Supplementation and Resting-state Brain Connectivity \\t RECRUITING \\

In [0]:
# Create rdd

rdd1 = sc.textFile(file)

rdd1.take(3)

Out[10]: ['NCT Number \\t Study Title \\t Acronym \\t Study Status \\t Conditions \\t Interventions \\t Sponsor \\t Collaborators \\t Enrollment \\t Funder Type \\t Study Type \\t Study Design,Start Date,Completion Date',
 '"NCT03630471 \\t Effectiveness of a Problem-solving Intervention for Common Adolescent Mental Health Problems in India \\t PRIDE \\t COMPLETED \\t Mental Health Issue (E.G., Depression, Psychosis, Personality Disorder, Substance Abuse) \\t BEHAVIORAL: PRIDE \'Step 1\' problem-solving intervention|BEHAVIORAL: Enhanced usual care \\t Sangath \\t Harvard Medical School (HMS and HSDM)|London School of Hygiene and Tropical Medicine \\t 250 \\t OTHER \\t INTERVENTIONAL \\t Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (INVESTIGATOR, OUTCOMES_ASSESSOR)|Primary Purpose: TREATMENT",20 August 2018,28 February 2019',
 '"NCT05992571 \\t Oral Ketone Monoester Supplementation and Resting-state Brain Connectivity \\t RECRUITING \\t Cerebrovascular Function|Co

In [0]:
#Extract header

header = rdd1.first()

print(header)

NCT Number \t Study Title \t Acronym \t Study Status \t Conditions \t Interventions \t Sponsor \t Collaborators \t Enrollment \t Funder Type \t Study Type \t Study Design,Start Date,Completion Date



Data Cleaning

In [0]:
#Let's split Header

# Extract header
header = rdd1.first()

# Split the header based on \t
header_elements = header.split("\t")

# Split the last element based on comma
last_element_split = header_elements[-1].split(',')

# Combine the split elements into the header list
header_elements[-1:] = last_element_split

# Print the modified header
print(header_elements)

['NCT Number \\t Study Title \\t Acronym \\t Study Status \\t Conditions \\t Interventions \\t Sponsor \\t Collaborators \\t Enrollment \\t Funder Type \\t Study Type \\t Study Design', 'Start Date', 'Completion Date']


In [0]:
#Define function to split lines in text

def split_line(line):
    # Split the line by tab character
    parts = line.split("\\t")
    # Split the last part by commas and extend the list
    last_part_split = parts[-1].split(",")
    return parts[:-1] + last_part_split

# Apply the split_line function to the input list and save as rdd2
rdd2 = rdd1.map(split_line)

rdd2.take(2)

Out[15]: [['NCT Number ',
  ' Study Title ',
  ' Acronym ',
  ' Study Status ',
  ' Conditions ',
  ' Interventions ',
  ' Sponsor ',
  ' Collaborators ',
  ' Enrollment ',
  ' Funder Type ',
  ' Study Type ',
  ' Study Design',
  'Start Date',
  'Completion Date'],
 ['"NCT03630471 ',
  ' Effectiveness of a Problem-solving Intervention for Common Adolescent Mental Health Problems in India ',
  ' PRIDE ',
  ' COMPLETED ',
  ' Mental Health Issue (E.G., Depression, Psychosis, Personality Disorder, Substance Abuse) ',
  " BEHAVIORAL: PRIDE 'Step 1' problem-solving intervention|BEHAVIORAL: Enhanced usual care ",
  ' Sangath ',
  ' Harvard Medical School (HMS and HSDM)|London School of Hygiene and Tropical Medicine ',
  ' 250 ',
  ' OTHER ',
  ' INTERVENTIONAL ',
  ' Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (INVESTIGATOR',
  ' OUTCOMES_ASSESSOR)|Primary Purpose: TREATMENT"',
  '20 August 2018',
  '28 February 2019']]

In [0]:
#Cleaning date
#Extract first line data
first_line = rdd2.map(lambda x: x[-1])

first_line.take(4)

Out[16]: ['Completion Date', '28 February 2019', '2024-08', '2014-12']

In [0]:
#Check the first row

Header_row = rdd2.take(1)

print(Header_row)

[['NCT Number ', ' Study Title ', ' Acronym ', ' Study Status ', ' Conditions ', ' Interventions ', ' Sponsor ', ' Collaborators ', ' Enrollment ', ' Funder Type ', ' Study Type ', ' Study Design', 'Start Date', 'Completion Date']]


In [0]:
# Extract Start Date and Completion Date

rdd3 = rdd2.map(lambda row: (row[-2], row[-1]))

rdd3.collect()  #show all data in rdd to examine date format that exist

Out[18]: [('Start Date', 'Completion Date'),
 ('20 August 2018', '28 February 2019'),
 ('25 October 2023', '2024-08'),
 ('2013-08', '2014-12'),
 ('01 October 2020', '01 October 2023'),
 ('01 July 2010', '30 October 2019'),
 ('05 May 2017', '09 October 2020'),
 ('01 September 2022', '15 June 2023'),
 ('01 February 2024', '01 March 2026'),
 ('11 May 2016', '13 December 2017'),
 ('2016-02', '2017-05'),
 ('13 July 2017', '30 June 2026'),
 ('2004-01', '2005-01'),
 ('09 February 2023', '17 November 2024'),
 ('2015-03', '2016-12'),
 ('13 June 2013', '02 November 2021'),
 ('13 June 2018', '24 December 2018'),
 ('2009-01', '2009-12'),
 ('2009-08', '2011-07'),
 ('2009-09', '2010-03'),
 ('02 December 2021', '31 December 2023'),
 ('2003-03', '2004-04'),
 ('30 May 2023', '30 May 2025'),
 ('15 July 2019', '01 September 2020'),
 ('15 April 2020', '31 May 2020'),
 ('2014-09', '2019-02'),
 ('01 July 2023', '01 December 2024'),
 ('01 January 2018', '31 December 2024'),
 ('06 May 2022', '06 November 2025

In [0]:
#Create date without the Header

#create Date Header

Date_Header = rdd3.first()

print(Date_Header)

('Start Date', 'Completion Date')


In [0]:
rdd4 = rdd3.map(lambda x: x[:2])

rdd4.take(3)

Out[20]: [('Start Date', 'Completion Date'),
 ('20 August 2018', '28 February 2019'),
 ('25 October 2023', '2024-08')]

In [0]:
#Function to clean the date and convert to a consistent date format

from datetime import datetime

# Define a function to clean date data
def clean_date(date_str):
    if not date_str:
        return ""
        
    formats = ['%d %B %Y', '%Y %m %d', '%d %m %Y', '%Y-%m']
    for fmt in formats:
        try:
            return datetime.strptime(date_str, fmt).strftime('%d %B %Y')
        except ValueError:
            pass
    return date_str

# Clean the date data in the RDD
cleaned_dates = rdd3.map(lambda x: (clean_date(x[0]), clean_date(x[1])))

# Collect and print the cleaned date data
for date_pair in cleaned_dates.collect():
    print(date_pair)

('Start Date', 'Completion Date')
('20 August 2018', '28 February 2019')
('25 October 2023', '01 August 2024')
('01 August 2013', '01 December 2014')
('01 October 2020', '01 October 2023')
('01 July 2010', '30 October 2019')
('05 May 2017', '09 October 2020')
('01 September 2022', '15 June 2023')
('01 February 2024', '01 March 2026')
('11 May 2016', '13 December 2017')
('01 February 2016', '01 May 2017')
('13 July 2017', '30 June 2026')
('01 January 2004', '01 January 2005')
('09 February 2023', '17 November 2024')
('01 March 2015', '01 December 2016')
('13 June 2013', '02 November 2021')
('13 June 2018', '24 December 2018')
('01 January 2009', '01 December 2009')
('01 August 2009', '01 July 2011')
('01 September 2009', '01 March 2010')
('02 December 2021', '31 December 2023')
('01 March 2003', '01 April 2004')
('30 May 2023', '30 May 2025')
('15 July 2019', '01 September 2020')
('15 April 2020', '31 May 2020')
('01 September 2014', '01 February 2019')
('01 July 2023', '01 December 202

In [0]:
## Create an rdd excluding the 'Start" and 'Completion" date

rdd5 = rdd2.map(lambda x: x[:11])

rdd2.take(10)

Out[41]: [['NCT Number ',
  ' Study Title ',
  ' Acronym ',
  ' Study Status ',
  ' Conditions ',
  ' Interventions ',
  ' Sponsor ',
  ' Collaborators ',
  ' Enrollment ',
  ' Funder Type ',
  ' Study Type ',
  ' Study Design',
  'Start Date',
  'Completion Date'],
 ['"NCT03630471 ',
  ' Effectiveness of a Problem-solving Intervention for Common Adolescent Mental Health Problems in India ',
  ' PRIDE ',
  ' COMPLETED ',
  ' Mental Health Issue (E.G., Depression, Psychosis, Personality Disorder, Substance Abuse) ',
  " BEHAVIORAL: PRIDE 'Step 1' problem-solving intervention|BEHAVIORAL: Enhanced usual care ",
  ' Sangath ',
  ' Harvard Medical School (HMS and HSDM)|London School of Hygiene and Tropical Medicine ',
  ' 250 ',
  ' OTHER ',
  ' INTERVENTIONAL ',
  ' Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (INVESTIGATOR',
  ' OUTCOMES_ASSESSOR)|Primary Purpose: TREATMENT"',
  '20 August 2018',
  '28 February 2019'],
 ['"NCT05992571 ',
  ' Oral Ketone Monoester Su

In [0]:
#Zip RDDs together and flatten the rdd

# Zip the RDDs together column-wise
zipped_rdd = rdd5.zip(cleaned_dates)

# Add start and completion date columns to each tuple in the zipped RDD as a tuple
flattened_rdd = zipped_rdd.map(lambda x: x[0] + list(x[1]))

# Output the result
flattened_rdd.take(20)

Out[23]: [['NCT Number ',
  ' Study Title ',
  ' Acronym ',
  ' Study Status ',
  ' Conditions ',
  ' Interventions ',
  ' Sponsor ',
  ' Collaborators ',
  ' Enrollment ',
  ' Funder Type ',
  ' Study Type ',
  ' Study Design',
  'Start Date',
  'Completion Date'],
 ['"NCT03630471 ',
  ' Effectiveness of a Problem-solving Intervention for Common Adolescent Mental Health Problems in India ',
  ' PRIDE ',
  ' COMPLETED ',
  ' Mental Health Issue (E.G., Depression, Psychosis, Personality Disorder, Substance Abuse) ',
  " BEHAVIORAL: PRIDE 'Step 1' problem-solving intervention|BEHAVIORAL: Enhanced usual care ",
  ' Sangath ',
  ' Harvard Medical School (HMS and HSDM)|London School of Hygiene and Tropical Medicine ',
  ' 250 ',
  ' OTHER ',
  ' INTERVENTIONAL ',
  ' Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (INVESTIGATOR',
  '20 August 2018',
  '28 February 2019'],
 ['"NCT05992571 ',
  ' Oral Ketone Monoester Supplementation and Resting-state Brain Connectivity ',

Exploratory data analysis

In [0]:
#Check the count of values by column header

# Extract the header row
header_row = flattened_rdd.first()

# Count the total number of non-empty values for each column
column_counts = flattened_rdd.flatMap(lambda row: zip(header_row, row)) \
                   .filter(lambda x: x[1] != '') \
                   .map(lambda x: (x[0], 1)) \
                   .reduceByKey(lambda a, b: a + b) \
                   .collect()

# Output the column counts
for column, count in column_counts:
    print(f"Column '{column}': {count}")

Column 'NCT Number ': 478698
Column ' Interventions ': 477801
Column ' Sponsor ': 477801
Column ' Collaborators ': 477775
Column ' Study Design': 467588
Column ' Study Title ': 478698
Column ' Study Status ': 478698
Column ' Funder Type ': 475705
Column ' Study Type ': 472627
Column 'Completion Date': 438184
Column ' Acronym ': 478697
Column ' Conditions ': 477815
Column ' Enrollment ': 477049
Column 'Start Date': 471145


In [0]:
#Likewise, Check missing values by column headers

# Extract the header row
header_row = flattened_rdd.first()

# Count the total number of empty values for each column
empty_counts = flattened_rdd.flatMap(lambda row: zip(header_row, row)) \
                  .filter(lambda x: x[1] == '') \
                  .map(lambda x: (x[0], 1)) \
                  .reduceByKey(lambda a, b: a + b) \
                  .collect()

# Output the column counts
for column, count in empty_counts:
    print(f"Column '{column}': {count}")

Column ' Study Design': 10187
Column ' Interventions ': 882
Column ' Sponsor ': 882
Column ' Collaborators ': 908
Column ' Study Type ': 5174
Column 'Completion Date': 12204
Column ' Funder Type ': 2096
Column 'Start Date': 5182
Column ' Enrollment ': 752
Column ' Conditions ': 883
Column ' Acronym ': 1


In [0]:
rdd1_splitline = rdd1 \
    .flatMap(lambda x: x.split('\n')) \
    .filter(lambda line: line.count("\\t") < 10)

rdd1_splitline.take(50)

Out[135]: ['NCT01160471 \\t Noninvasive Imaging of Heart Failure: A Pilot Study \\t COMPLETED \\t MRI|Acquired Heart Disease|Myocardial Fibrosis \\t National Institutes of Health Clinical Center (CC) \\t 62 \\t NIH \\t OBSERVATIONAL \\t Observational Model: |Time Perspective: p,01 July 2010,30 October 2019',
 '"NCT04338971 \\t Ipsihand Device Use in Stroke Patients to Assess Functional Motor Outcomes \\t COMPLETED \\t Stroke \\t DEVICE: IpsiHand Therapy \\t Neurolutions, Inc. \\t 24 \\t INDUSTRY \\t INTERVENTIONAL \\t Allocation: NA|Intervention Model: SINGLE_GROUP|Masking: NONE|Primary Purpose: TREATMENT",05 May 2017,09 October 2020',
 '"NCT05843071 \\t PMCF Study to Evaluate Performance and Safety of ""NASAL SPRAYS "" Used to Relieve Nasal Congestion and Dryness \\t RECRUITING \\t Nasal Congestion|Nasal Dryness \\t DEVICE: Nasal sprays with sea salts \\t C.O.C. Farmaceutici S.r.l. \\t 105 \\t INDUSTRY \\t INTERVENTIONAL \\t Allocation: NA|Intervention Model: SINGLE_GROUP|Masking: NON

In [0]:
# Apply a FlatMap operation to extract all conditions

rdd_condition = rdd1 \
    .flatMap(lambda x: x.split('\n'))
rdd_condition.take(20)

Out[31]: ['NCT Number \\t Study Title \\t Acronym \\t Study Status \\t Conditions \\t Interventions \\t Sponsor \\t Collaborators \\t Enrollment \\t Funder Type \\t Study Type \\t Study Design,Start Date,Completion Date',
 '"NCT03630471 \\t Effectiveness of a Problem-solving Intervention for Common Adolescent Mental Health Problems in India \\t PRIDE \\t COMPLETED \\t Mental Health Issue (E.G., Depression, Psychosis, Personality Disorder, Substance Abuse) \\t BEHAVIORAL: PRIDE \'Step 1\' problem-solving intervention|BEHAVIORAL: Enhanced usual care \\t Sangath \\t Harvard Medical School (HMS and HSDM)|London School of Hygiene and Tropical Medicine \\t 250 \\t OTHER \\t INTERVENTIONAL \\t Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (INVESTIGATOR, OUTCOMES_ASSESSOR)|Primary Purpose: TREATMENT",20 August 2018,28 February 2019',
 '"NCT05992571 \\t Oral Ketone Monoester Supplementation and Resting-state Brain Connectivity \\t RECRUITING \\t Cerebrovascular Function|Co

In [0]:
rdd_condition = rdd1 \
    .flatMap(lambda x: x.split('\n')) \
    .filter(lambda line: line.count("\\t") > 10)

rdd_condition.take(10)

Out[32]: ['NCT Number \\t Study Title \\t Acronym \\t Study Status \\t Conditions \\t Interventions \\t Sponsor \\t Collaborators \\t Enrollment \\t Funder Type \\t Study Type \\t Study Design,Start Date,Completion Date',
 '"NCT03630471 \\t Effectiveness of a Problem-solving Intervention for Common Adolescent Mental Health Problems in India \\t PRIDE \\t COMPLETED \\t Mental Health Issue (E.G., Depression, Psychosis, Personality Disorder, Substance Abuse) \\t BEHAVIORAL: PRIDE \'Step 1\' problem-solving intervention|BEHAVIORAL: Enhanced usual care \\t Sangath \\t Harvard Medical School (HMS and HSDM)|London School of Hygiene and Tropical Medicine \\t 250 \\t OTHER \\t INTERVENTIONAL \\t Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (INVESTIGATOR, OUTCOMES_ASSESSOR)|Primary Purpose: TREATMENT",20 August 2018,28 February 2019',
 '"NCT03400371 \\t Biology of Juvenile Myoclonic Epilepsy \\t BIOJUME \\t RECRUITING \\t Juvenile Myoclonic Epilepsy \\t OTHER: Blood draw|O

In [0]:
# Extract the header row
header_row = flattened_rdd.first()

# Find the index of the 'Study Title' column
study_title_index = header_row.index(' Study Title ')

# Count the number of distinct studies
distinct_study_count = flattened_rdd.map(lambda row: row[study_title_index]) \
                          .distinct() \
                          .count()

# Output the number of distinct studies
print("Number of distinct studies:", distinct_study_count)

Number of distinct studies: 476396


In [0]:
#Checking the count of length of each line after split so that we don't lose on data
types = rdd1 \
    .map(lambda x: x.split("\\t")) \

# Initialize counters
len_9_count = 0
len_10_count = 0
len_11_count = 0
len_12_count = 0

# Print the length of each line for 50 lines after the split
for i, line in enumerate(types.collect(), 1):
    print(f"Line {i}: Length = {len(line)}")
    if len(line) == 9:
        len_9_count += 1
    elif len(line) == 10:
        len_10_count += 1
    elif len(line) == 11:
        len_11_count += 1
    elif len(line) == 12:
        len_12_count += 1

# Print total counts
print(f"Total number of lines with length 9: {len_9_count}")
print(f"Total number of lines with length 10: {len_10_count}")
print(f"Total number of lines with length 11: {len_11_count}")
print(f"Total number of lines with length 12: {len_12_count}")

#It was found that after split, the lines have between 9-12 length. Majority of the lines have length 10 & 11

Line 1: Length = 12
Line 2: Length = 12
Line 3: Length = 11
Line 4: Length = 11
Line 5: Length = 11
Line 6: Length = 9
Line 7: Length = 10
Line 8: Length = 10
Line 9: Length = 9
Line 10: Length = 11
Line 11: Length = 11
Line 12: Length = 12
Line 13: Length = 9
Line 14: Length = 10
Line 15: Length = 12
Line 16: Length = 10
Line 17: Length = 11
Line 18: Length = 12
Line 19: Length = 10
Line 20: Length = 10
Line 21: Length = 11
Line 22: Length = 11
Line 23: Length = 10
Line 24: Length = 11
Line 25: Length = 10
Line 26: Length = 11
Line 27: Length = 10
Line 28: Length = 10
Line 29: Length = 9
Line 30: Length = 9
Line 31: Length = 10
Line 32: Length = 11
Line 33: Length = 11
Line 34: Length = 11
Line 35: Length = 10
Line 36: Length = 10
Line 37: Length = 10
Line 38: Length = 11
Line 39: Length = 11
Line 40: Length = 10
Line 41: Length = 11
Line 42: Length = 10
Line 43: Length = 10
Line 44: Length = 10
Line 45: Length = 11
Line 46: Length = 11
Line 47: Length = 11
Line 48: Length = 10
Line 4

In [0]:
#First Attempt 1 on Question 2 using only lines with length 10, study types index position in this case is 8

types = rdd1_splitline \
    .map(lambda x: x.split("\\t")) \
    .filter(lambda x: len(x) == 10) \
    .map(lambda x: x[8]) \
    .flatMap(lambda x: x.split("|")) \
    .filter(lambda x: x.strip() != 'Allocated') \
    .filter(lambda x: x.strip() != 'INDUSTRY') \
    .filter(lambda x: x.strip() != 'FED') \
    .map(lambda x: (x.strip(), 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .map(lambda x: (x[1], x[0])) \
    .sortByKey(ascending=False) \
    .map(lambda x: (x[1], x[0]))


# Output the top 5
study_types = types.take(5)

for types, frequency in study_types:
    print(f"Study_Type: {types}, Frequency: {frequency}")


Study_Type: INTERVENTIONAL, Frequency: 186900
Study_Type: OBSERVATIONAL, Frequency: 47673
Study_Type: OTHER, Frequency: 14


In [0]:
#Using all lines in the dataset and adjusting the index position to capture study types.
# At the end, I filter values that are not study types

# For lines with length 10
types_len_10 = rdd1_splitline \
    .map(lambda x: x.split("\\t")) \
    .filter(lambda x: len(x) == 10) \
    .map(lambda x: x[8]) \
    .flatMap(lambda x: x.split("|")) \
    .filter(lambda x: x.strip() != 'Allocated') \
    .map(lambda x: (x.strip(), 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .map(lambda x: (x[1], x[0])) \
    .sortByKey(ascending=False) \
    .map(lambda x: (x[1], x[0]))

# For lines with length 9
types_len_9 = rdd1_splitline \
    .map(lambda x: x.split("\\t")) \
    .filter(lambda x: len(x) == 9) \
    .map(lambda x: x[7]) \
    .flatMap(lambda x: x.split("|")) \
    .filter(lambda x: x.strip() != 'Allocated') \
    .map(lambda x: (x.strip(), 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .map(lambda x: (x[1], x[0])) \
    .sortByKey(ascending=False) \
    .map(lambda x: (x[1], x[0]))

# For lines with length 11
types_len_11 = rdd1_splitline \
    .map(lambda x: x.split("\\t")) \
    .filter(lambda x: len(x) == 11) \
    .map(lambda x: x[9]) \
    .flatMap(lambda x: x.split("|")) \
    .filter(lambda x: x.strip() != 'Allocated') \
    .map(lambda x: (x.strip(), 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .map(lambda x: (x[1], x[0])) \
    .sortByKey(ascending=False) \
    .map(lambda x: (x[1], x[0]))

# For lines with length 12
types_len_12 = rdd1_splitline \
    .map(lambda x: x.split("\\t")) \
    .filter(lambda x: len(x) == 12) \
    .map(lambda x: x[10]) \
    .flatMap(lambda x: x.split("|")) \
    .filter(lambda x: x.strip() != 'Allocated') \
    .map(lambda x: (x.strip(), 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .map(lambda x: (x[1], x[0])) \
    .sortByKey(ascending=False) \
    .map(lambda x: (x[1], x[0]))

# Combine results for all lengths
all_types = types_len_10.union(types_len_9).union(types_len_11).union(types_len_12)

# Aggregate frequencies for each study type
aggregated_types = all_types.reduceByKey(lambda x, y: x + y)

# Sort aggregated types by frequency in ascending order
sorted_types = aggregated_types.sortBy(lambda x: x[1], ascending=False)

# Output the sorted results
for study_type, frequency in sorted_types.take(3):
    print(f"Study_Type: {study_type}, Frequency: {frequency}")

Study_Type: INTERVENTIONAL, Frequency: 189457
Study_Type: OBSERVATIONAL, Frequency: 70860
Study_Type: OTHER, Frequency: 132


In [0]:
#Question 3: The Top 5 Conditions with their frequencies and filter out values that are not condition

top5 = rdd1_splitline \
    .map(lambda x: x.split("\\t")) \
    .filter(lambda x: len(x) > 3) \
    .map(lambda x: x[3]) \
    .flatMap(lambda x: x.split("|")) \
    .filter(lambda x: x.strip() != 'COMPLETED') \
    .map(lambda x: (x.strip(), 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .map(lambda x: (x[1], x[0])) \
    .sortByKey(ascending=False) \
    .map(lambda x: (x[1], x[0]))


# Output the top 5
top_5_conditions = top5.take(5)

for condition, frequency in top_5_conditions:
    print(f"Condition: {condition}, Frequency: {frequency}")


Condition: Healthy, Frequency: 7003
Condition: Breast Cancer, Frequency: 3454
Condition: Obesity, Frequency: 2863
Condition: Pain, Frequency: 2379
Condition: HIV Infections, Frequency: 1992


In [0]:
#Pharma unzipped, Let's view the file

pharma_file = "/FileStore/tables/pharma/pharma.csv"

dbutils.fs.head(pharma_file)

[Truncated to first 65536 bytes]
Out[157]: '"Company","Parent_Company","Penalty_Amount","Subtraction_From_Penalty","Penalty_Amount_Adjusted_For_Eliminating_Multiple_Counting","Penalty_Year","Penalty_Date","Offense_Group","Primary_Offense","Secondary_Offense","Description","Level_of_Government","Action_Type","Agency","Civil/Criminal","Prosecution_Agreement","Court","Case_ID","Private_Litigation_Case_Title","Lawsuit_Resolution","Facility_State","City","Address","Zip","NAICS_Code","NAICS_Translation","HQ_Country_of_Parent","HQ_State_of_Parent","Ownership_Structure","Parent_Company_Stock_Ticker","Major_Industry_of_Parent","Specific_Industry_of_Parent","Info_Source","Notes"\n"Abbott Laboratories","Abbott Laboratories","$5,475,000","$0","$5,475,000","2013","20131227","government-contracting-related offenses","False Claims Act and related","kickbacks and bribery","Abbott Laboratories agreed to $5.475 million to resolve allegations that it violated the False Claims Act by paying kickbacks to i

In [0]:
#Create rdd

pharma_rdd = sc.textFile(pharma_file)

pharma_rdd.take(3)

Out[158]: ['"Company","Parent_Company","Penalty_Amount","Subtraction_From_Penalty","Penalty_Amount_Adjusted_For_Eliminating_Multiple_Counting","Penalty_Year","Penalty_Date","Offense_Group","Primary_Offense","Secondary_Offense","Description","Level_of_Government","Action_Type","Agency","Civil/Criminal","Prosecution_Agreement","Court","Case_ID","Private_Litigation_Case_Title","Lawsuit_Resolution","Facility_State","City","Address","Zip","NAICS_Code","NAICS_Translation","HQ_Country_of_Parent","HQ_State_of_Parent","Ownership_Structure","Parent_Company_Stock_Ticker","Major_Industry_of_Parent","Specific_Industry_of_Parent","Info_Source","Notes"',
 '"Abbott Laboratories","Abbott Laboratories","$5,475,000","$0","$5,475,000","2013","20131227","government-contracting-related offenses","False Claims Act and related","kickbacks and bribery","Abbott Laboratories agreed to $5.475 million to resolve allegations that it violated the False Claims Act by paying kickbacks to induce doctors to implant the 

In [0]:
#Split by line and delimiter

pharma_rdd1 = pharma_rdd \
    .flatMap(lambda x: x.split('\n')) \
    .map(lambda line: line.split(","))

pharma_rdd1.take(10)

Out[160]: [['"Company"',
  '"Parent_Company"',
  '"Penalty_Amount"',
  '"Subtraction_From_Penalty"',
  '"Penalty_Amount_Adjusted_For_Eliminating_Multiple_Counting"',
  '"Penalty_Year"',
  '"Penalty_Date"',
  '"Offense_Group"',
  '"Primary_Offense"',
  '"Secondary_Offense"',
  '"Description"',
  '"Level_of_Government"',
  '"Action_Type"',
  '"Agency"',
  '"Civil/Criminal"',
  '"Prosecution_Agreement"',
  '"Court"',
  '"Case_ID"',
  '"Private_Litigation_Case_Title"',
  '"Lawsuit_Resolution"',
  '"Facility_State"',
  '"City"',
  '"Address"',
  '"Zip"',
  '"NAICS_Code"',
  '"NAICS_Translation"',
  '"HQ_Country_of_Parent"',
  '"HQ_State_of_Parent"',
  '"Ownership_Structure"',
  '"Parent_Company_Stock_Ticker"',
  '"Major_Industry_of_Parent"',
  '"Specific_Industry_of_Parent"',
  '"Info_Source"',
  '"Notes"'],
 ['"Abbott Laboratories"',
  '"Abbott Laboratories"',
  '"$5',
  '475',
  '000"',
  '"$0"',
  '"$5',
  '475',
  '000"',
  '"2013"',
  '"20131227"',
  '"government-contracting-related of

In [0]:
#Extract Parent Company
pharma_rdd2 = pharma_rdd1 \
    .map(lambda x: x[1][1:-1]) \
    .filter(lambda x: x.strip() != "")

pharma_rdd2.take(5)


Out[172]: ['Parent_Company', 'Abbott Laboratories', 'AbbVie', 'AbbVie', 'Inc.']

In [0]:
#From rdd1 extract NCT Number and sponsor

rdd_sponsor = rdd1 \
    .flatMap(lambda x: x.split('\n')) \
    .map(lambda line: line.split("\\t")) \
    .map(lambda x: (x[6], x[0]))

# Print the first few elements to verify the extraction
rdd_sponsor.take(5)

Out[170]: [(' Sponsor ', 'NCT Number '),
 (' Sangath ', '"NCT03630471 '),
 (" Alzheimer's Society of Brant, Haldimand Norfolk, Hamilton Halton ",
  '"NCT05992571 '),
 (' Society for Family Planning Research Fund ', 'NCT01854671 '),
 (' Assistance Publique - Hôpitaux de Paris ', 'NCT03820271 ')]